<a href="https://colab.research.google.com/github/marco-luzzara/boardgame-complexity-predictor/blob/master/src/extract_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython.display import clear_output
import os
WORKING_LOCALLY = bool(os.getenv('WORKING_LOCALLY'))

if WORKING_LOCALLY:
    DATASET_FILE_PATH = 'data/dataset.csv'
    CLEANED_DATASET_FILE_PATH = 'data/cleaned_dataset.csv'
else:
    from google.colab import drive
    drive.mount('/content/drive')
    DATASET_FILE_PATH = '/content/drive/My Drive/Projects/IRBoardGameComplexity/dataset.csv'
    CLEANED_DATASET_FILE_PATH = '/content/drive/My Drive/Projects/IRBoardGameComplexity/cleaned_dataset.csv'
    # !pip install git+https://github.com/LIAAD/yake
    # !pip install rake-nltk
    clear_output(wait=False)

In [3]:
from IPython.display import clear_output
import spacy
from spacy import displacy

In [4]:
import logging

logger = logging.getLogger('bgg_predict')
logger.handlers.clear()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

logger.debug('test')

2022-11-28 19:27:46,701 bgg_predict  DEBUG    test
DEBUG:bgg_predict:test


In [5]:
import pandas as pd
import ast

def get_df_with_docs(file_path: str, nrows=None, skiprows=1) -> pd.DataFrame:
    ''' get a dataframe containing nrows and skipping the first `skiprows` (including the header)'''
    df_dataset = pd.read_csv(file_path, converters={ 'family': ast.literal_eval }, 
                             nrows=nrows, skiprows=range(1, skiprows))
    return df_dataset

def get_document_by_line(file_path: str, line: int) -> str:
    ''' the line includes the header too '''
    # range from 1 is used to keep the first row https://stackoverflow.com/a/27325729/5587393
    df = get_df_with_docs(file_path, 1, line - 1)
    return df['rulebook'].iloc[0]

def get_document_by_id(file_path: str, id: int) -> str:
     with pd.read_csv(file_path, chunksize=1, converters={ 'family': ast.literal_eval }) as reader:
        while True:
            df = next(reader)
            bg_id = df['id'].iloc[0]
            if bg_id == id:
                return df['rulebook'].iloc[0]

assert get_document_by_id(CLEANED_DATASET_FILE_PATH, 2310) == get_document_by_line(CLEANED_DATASET_FILE_PATH, 40)

## Cleaning and Preprocessing

In this part, data are cleaned and processed using coreference resolution. This means that all the pronouns and references to other objects in the sentence are resolved. The next 2 cells should be run only when you want to preprocess data, which takes a lot of time.

In [ ]:
if not WORKING_LOCALLY:
    !pip install spacy-transformers
    !python3 -m pip install coreferee==1.3.*
    !python3 -m coreferee install en
    !python -m spacy download en_core_web_lg
    !python -m spacy download en_core_web_trf
    clear_output(wait=False)

In [ ]:
import re

regex_mail = re.compile(r'\w+(?:\.\w+)*?@\w+(?:\.\w+)+')
# modified from https://stackoverflow.com/a/163684/5587393
regex_link = re.compile(r'(?:\b(?:(?:https?|ftp|file)://|www))[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#%=~_|]')
# in a sentence there must be at least 4 words of length 2 each
regex_at_least_4_words_in_sentence = re.compile(r"^(?=.*?(?:[,:;()'\"]?[a-zA-Z']{2,}[,:;()'\"]?(?: |-|$)(?:[^a-zA-Z]*?|[a-zA-Z]? ?)){4,})")         
# a string like "first.Second" could be misinterpreted by the tokenizer as a single token
# with the regex it becomes "first. Second"
regex_distance_between_period_and_following_word = re.compile(r'\.(?!\s|$)')
# compress consecutive whitespaces
regex_multiple_spaces = re.compile(r'\s{2,}')
# interrupted words usually have a "- " at the end before the new line, 'inter- rupted' -> 'interrupted'
# NOTE: must be after whitespace compression
regex_interrupted_word = re.compile(r'([a-zA-Z])- ')
# remove page numbers, that are usually enclosed in characters like = or -, for example "-12-"
regex_consecutive_meaningless_chars = re.compile(r'[^\.a-zA-Z0-9\s()]{2,} *(?:\d+)?|(?P<prepage>[^a-zA-Z\s\d\.])\d+(?P=prepage)')
# remove paragraphs id, '1.2.3' -> ''
regex_dot_separated_digits = re.compile(r'(?:\d+\.)+\d+')
# remove meaningless chars after sentence start, '. (- start' -> '. start'
regex_clean_start = re.compile(r'\.(\s?)[^a-zA-Z\s]+')
# recover missing apices
regex_missing_apices = re.compile(r"\b([a-zA-Z]+) (t|s)\b")

def clean_from_short_sentences(text: str) -> str:
    return '.'.join(sentence for sentence in text.split('.') if regex_at_least_4_words_in_sentence.match(sentence) is not None)

def clean_text(text: str) -> str:
    for clean_function in [lambda x: regex_mail.sub('', x),
                           lambda x: regex_link.sub('', x),
                           lambda x: regex_dot_separated_digits.sub('', x),
                           lambda x: regex_consecutive_meaningless_chars.sub('', x),
                           lambda x: regex_clean_start.sub(r'.\1', x),
                           # everything that is remove should be placed before this line so that 
                           # eventual spaces are compressed with regex_multiple_space
                           lambda x: regex_multiple_spaces.sub(' ', x),
                           lambda x: regex_interrupted_word.sub(r'\1', x),
                           lambda x: regex_missing_apices.sub(r"\1'\2", x),
                           lambda x: clean_from_short_sentences(x),
                           lambda x: regex_distance_between_period_and_following_word.sub('. ', x)]:
        text = clean_function(text)
    return text

test_text = 'this is a test (me@gmail.it) -12- that wi-  ll be   cleaned. with 2 5 6 not valid. two sentences can t be good http://or.not.'
cleaned_text = clean_text(test_text)
print(cleaned_text)
assert cleaned_text == 'this is a test () that will be cleaned. two sentences can\'t be good '

this is a test () that will be cleaned. two sentences can't be good 


In [ ]:
from typing import List
import pandas as pd
import coreferee

def remove_columns_prefix(df: pd.DataFrame) -> None:
    '''remove prefix 'info.' from the columns of df'''
    df.rename(columns=lambda c: c.rsplit('.', 1)[-1], inplace=True)

def _get_new_token_from_resolve(token: spacy.tokens.Token, 
                                chains: coreferee.data_model.ChainHolder) -> spacy.tokens.Token:
    resolved_token = chains.resolve(token)
    return token.text_with_ws if resolved_token is None \
                              else 'and '.join([res_token.text_with_ws + ' ' for res_token in resolved_token])   

def _process_doc_for_coref(doc: spacy.tokens.Doc) -> str:
    replacement_tokens = []
    chains = doc._.coref_chains
    new_doc_tokens_text = [_get_new_token_from_resolve(token, chains) for token in doc]

    return ''.join(new_doc_tokens_text)

def preprocess_texts(texts: List[str]) -> List[str]:
    nlp = spacy.load('en_core_web_trf')
    nlp.add_pipe("coreferee")

    texts = [clean_text(text) for text in texts]
    docs = nlp.pipe(texts)

    return [regex_multiple_spaces.sub(' ', _process_doc_for_coref(doc)) for doc in docs]
        
text = get_document_by_line(DATASET_FILE_PATH, 103)
# text = '''Although he was very busy with his work, the magical Peter had had enough of it. 
#     He and his wife decided they needed a holiday. 
#     this couple travelled to Spain because it loves the country very much.'''
preprocess_texts([text])

In [ ]:
import pandas as pd
from typing import Generator, Tuple
import ast

START_CLEANING = False
CHUNK_SIZE = 20
DATASET_ROWS = 381

assert START_CLEANING == True # make sure you do not start preprocessing again

def clean_data_row(row, docs_dict: Generator[Tuple[int, str], None, None]):
    id_rulebook = next(docs_dict)
    assert id_rulebook[0] == row['id']
    row['rulebook'] = id_rulebook[1]
    return row

drive.mount('/content/drive')
for skip_rows in range(1, DATASET_ROWS, CHUNK_SIZE):
    column_names = ['rulebook', 'info.id', 'info.name', 'info.averageweight', 'info.playingtime', 'info.family']
    # ast.literal_eval converts the family column string into a python array
    df_dataset = pd.read_csv(DATASET_FILE_PATH, converters={ 'info.family': ast.literal_eval },
                            names=column_names, header=None,
                            nrows=CHUNK_SIZE, skiprows=skip_rows)
    remove_columns_prefix(df_dataset)
    logger.info(f"processing boardgames from {df_dataset.loc[0, 'id']} to {df_dataset.loc[df_dataset.index[-1], 'id']}")
    docs_dict = zip(df_dataset['id'].values, preprocess_texts(df_dataset['rulebook'].values))

    df_cleaned_dataset = df_dataset.apply(lambda x: clean_data_row(x, docs_dict),
                                        axis='columns')

    df_cleaned_dataset.to_csv(CLEANED_DATASET_FILE_PATH, 
                            header=True if skip_rows == 1 else False, index=False, 
                            mode='w' if skip_rows == 1 else 'a')

if not WORKING_LOCALLY:
    drive.flush_and_unmount()
drive.mount('/content/drive')
df_cleaned_dataset.head()

## Luck metrics
these metrics are retrieved using rule-based matching and dependency matching. Luck is one of the criteria that determine the bg weight. In this case, the sources of luck considered are:

- Dice rolling
- Drawing
- Shuffling
- Words like *random* or *randomly*

In [ ]:
from spacy import displacy
from collections import namedtuple
from spacy.matcher import Matcher, DependencyMatcher

# TODO: flip a coin, flip sth like a coin
LuckMetrics = namedtuple('LuckMetrics', ['dice_based', 'drawing_based', 'shuffling_based', 'random_based'])

def get_luck_metrics(doc: spacy.tokens.Doc) -> LuckMetrics:
    # ---------- random ----------
    random_matcher = Matcher(doc.vocab)
    random_patterns_match = [
        [{"LEMMA": { "IN": ["random", "randomly"]}}]
    ]
    random_matcher.add("random", random_patterns_match)

    # ---------- shuffle ----------
    shuffle_matcher = Matcher(doc.vocab)
    shuffle_patterns_match = [
        [{"LEMMA": "shuffle"}]
    ]
    shuffle_matcher.add("shuffle", shuffle_patterns_match)

    # ---------- card drawing ----------
    drawing_matcher = DependencyMatcher(doc.vocab)    
    drawing_patterns = [
        [
            {
                "RIGHT_ID": "drawing",
                "RIGHT_ATTRS": {"LEMMA": "draw", "POS": "VERB"}
            },
            {
                "LEFT_ID": "drawing",
                "REL_OP": ">",
                "RIGHT_ID": "card",
                "RIGHT_ATTRS": {
                    "LEMMA": "card",
                    "POS": "NOUN", 
                    "DEP": { "IN": ['dobj', 'nsubjpass', 'compound'] }
                }
            }
        ]
    ]
    drawing_matcher.add("drawing", drawing_patterns)
    # ---------- dice rolling ----------
    dice_matcher = DependencyMatcher(doc.vocab)    
    dice_patterns = [
        [
            {
                "RIGHT_ID": "rolling",
                "RIGHT_ATTRS": {"LEMMA": { "IN": ["use", "throw", "roll"]}, "POS": "VERB"}
            },
            {
                "LEFT_ID": "rolling",
                "REL_OP": ">",
                "RIGHT_ID": "dice_or_die",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["die", "dice"]},
                    "POS": "NOUN", 
                    "DEP": { "IN": ['nsubj', 'dobj', 'nsubjpass', 'compound'] }
                }
            }
        ],
        [
            {
                "RIGHT_ID": "rolling",
                "RIGHT_ATTRS": {"LEMMA": { "IN": ["use", "throw", "roll"]}, "POS": "VERB"}
            },
            {
                "LEFT_ID": "rolling",
                "REL_OP": ">",
                "RIGHT_ID": "number",
                "RIGHT_ATTRS": {
                    "IS_DIGIT": True, 
                    "DEP": { "IN": ['dobj'] }
                }
            }
        ]
    ]
    dice_matcher.add("diceroll", dice_patterns)

    dice_matches = dice_matcher(doc) 
    draw_matches = drawing_matcher(doc)
    shuffle_matches = shuffle_matcher(doc)
    random_matches = random_matcher(doc)

    # TODO: needs normalization? (divide by rulebook length or tokens)

    return LuckMetrics(len(dice_matches), len(draw_matches), len(shuffle_matches), len(random_matches))

text = get_document_by_line(CLEANED_DATASET_FILE_PATH, 130)

nlp = spacy.load('en_core_web_sm')
doc = nlp(clean_text(text))
print(len(doc), len(doc.text))
print(get_luck_metrics(doc))

# displacy.render(doc, style='dep', jupyter=True)

2607 12624
LuckMetrics(dice_based=12, drawing_based=4, shuffling_based=3, random_based=0)


In [ ]:
# text = '''you can only take this because it can be outrageous. 
#     you can't take it. you could not also choose. you may never be sure of the result. 
#     you can decide the next thing. he has no other choice but to stop, another option is winning.'''

text = '''In some cases Good and Goods could have different lemma'''

nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
print([token.pos for token in doc])
print(len(doc), len(doc.text))

displacy.render(doc, style='dep', jupyter=True)

[85, 90, 92, 84, 89, 96, 87, 100, 84, 92]
10 55


## Amount of choices metrics
these metrics are retrieved using rule-based matching and dependency matching. The amount of choices every player has is one of the criteria that determine the bg weight. In this case, I am considering:

- *can/could/may/decide/...*, with some exceptions:
    - negatives are not considered choices, like *cannot draw* or *don't choose*
    - *can* + *choose* and similar ones increase the *amount of choices* metrics by 1
- *choice/option*, except when there is a leading *no*.

In [ ]:
from spacy import displacy
from collections import namedtuple
from spacy.matcher import Matcher, DependencyMatcher

def get_choices_amount_metric(doc: spacy.tokens.Doc) -> int:
    # --------------  can/could/may/choose/select/... -------------- 
    # all can/could/may
    can_could_may_matcher = Matcher(doc.vocab)
    can_could_may_patterns = [
        [{
            "LEMMA": { "IN": ["can", "could", "may", "decide", "select", "choose", "opt"]}, 
            "POS": { "IN": ["AUX", "VERB"]}
        }]
    ]
    can_could_may_matcher.add('can_could_may', can_could_may_patterns)
    can_could_may_matches = { match[1] for match in can_could_may_matcher(doc) }

    # can/could/may with only or neg
    can_could_may_exceptions_matcher = DependencyMatcher(doc.vocab)
    can_could_may_exceptions_patterns = [
        [
            # ❌ can not/only/never verb 
            {
                "RIGHT_ID": "can_could_may",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["can", "could", "may"]}, 
                    "POS": "AUX"
                }
            },
            {
                "LEFT_ID": "can_could_may",
                "REL_OP": "<",
                "RIGHT_ID": "generic_verb",
                "RIGHT_ATTRS": {
                    "POS": { "IN": ["AUX", "VERB"] }
                }
            },
            {
                "LEFT_ID": "generic_verb",
                "REL_OP": ">",
                "RIGHT_ID": "neg_or_only",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["not", "only", "never"]}, 
                    "DEP": { "IN": ["advmod", "neg"] }
                }
            }
        ],
        [
            # ❌ not/only/never choose
            {
                "RIGHT_ID": "decision_verb",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["decide", "select", "choose", "opt"]}, 
                    "POS": "VERB"
                }
            },
            {
                "LEFT_ID": "decision_verb",
                "REL_OP": ">",
                "RIGHT_ID": "negation",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["not", "only", "never"]}, 
                    "DEP": { "IN": ["advmod", "neg"] }
                }
            }
        ],
        [
            # ❌ can + choose are counted as 1. can token is left out
            {
                "RIGHT_ID": "can_could_may",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["can", "could", "may"]}, 
                    "POS": "AUX"
                }
            },
            {
                "LEFT_ID": "can_could_may",
                "REL_OP": "<",
                "RIGHT_ID": "decision_verb",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["decide", "select", "choose", "opt"]},
                    "POS": "VERB"
                }
            }
        ]
    ]
    can_could_may_exceptions_matcher.add('can_could_may_exceptions', can_could_may_exceptions_patterns)
    can_could_may_exceptions_matches = { match[1][0] for match in can_could_may_exceptions_matcher(doc) }

    # -------------- choice and option -------------- 
    choice_option_matcher = Matcher(doc.vocab)
    choice_option_patterns = [
        [{
            "LEMMA": { "IN": ["choice", "option"]}, 
            "POS": "NOUN"
        }]
    ]
    choice_option_matcher.add('choice_option', choice_option_patterns)
    choice_option_matches = { match[1] for match in choice_option_matcher(doc) }

    choice_option_exceptions_matcher = DependencyMatcher(doc.vocab)
    choice_option_exceptions_patterns = [
        [
            {
                "RIGHT_ID": "choice",
                "RIGHT_ATTRS": {
                    "LEMMA": { "IN": ["choice", "option"]}, 
                    "POS": "NOUN"
                }
            },
            {
                "LEFT_ID": "choice",
                "REL_OP": ">",
                "RIGHT_ID": "prefix_no",
                "RIGHT_ATTRS": {
                    "LEMMA": "no",
                    "POS": "DET",
                    "DEP": "det"
                }
            }
        ]
    ]
    choice_option_exceptions_matcher.add('choice_option_exceptions', choice_option_exceptions_patterns)
    choice_option_exceptions_matches = { match[1][0] for match in choice_option_exceptions_matcher(doc) }

    return len(can_could_may_matches.difference(can_could_may_exceptions_matches)) + \
           len(choice_option_matches.difference(choice_option_exceptions_matches))

text = get_document_by_line(CLEANED_DATASET_FILE_PATH, 130)
# text = '''you can only take this because it can be outrageous. 
#     you can't take it. you can not also choose. you can never be sure of the result. 
#     you can decide the next thing, or you choose the target. another choice is to win. 
#     but there is no right option.'''

nlp = spacy.load('en_core_web_sm')
doc = nlp(clean_text(text))
print([(i, token.lemma_) for i, token in enumerate(doc)])
print(len(doc), len(doc.text))
print(get_choices_amount_metric(doc))

# displacy.render(doc, style='dep', jupyter=True)

[(0, 'page'), (1, '1'), (2, 'of'), (3, '4'), (4, '"'), (5, 'wizardology'), (6, '"'), (7, 'GAME'), (8, 'play'), (9, 'objective'), (10, ':'), (11, 'to'), (12, 'become'), (13, 'a'), (14, 'Master'), (15, 'Wizard'), (16, 'by'), (17, 'collect'), (18, '4'), (19, 'talisman'), (20, '('), (21, 'a'), (22, 'familiar'), (23, ','), (24, 'a'), (25, 'wizard'), (26, "'s"), (27, 'hat'), (28, ','), (29, 'a'), (30, 'staff'), (31, ','), (32, 'and'), (33, 'an'), (34, 'amulet'), (35, ')'), (36, 'and'), (37, 'free'), (38, 'Merlin'), (39, "'s"), (40, 'spirit'), (41, 'from'), (42, 'the'), (43, 'old'), (44, 'oak'), (45, 'tree'), (46, '.'), (47, 'place'), (48, 'the'), (49, 'maze'), (50, 'board'), (51, 'in'), (52, 'the'), (53, 'center'), (54, 'of'), (55, 'the'), (56, 'table'), (57, '.'), (58, 'place'), (59, 'a'), (60, 'Spirit'), (61, 'Chamber'), (62, 'at'), (63, 'each'), (64, 'of'), (65, 'the'), (66, '4'), (67, 'open'), (68, 'outer'), (69, 'doorway'), (70, 'of'), (71, 'the'), (72, 'game'), (73, 'board'), (74, '.')

In [39]:
text = '''You can only load onto the truck's white area.'''

# text = get_document_by_line(CLEANED_DATASET_FILE_PATH, 130)

nlp = spacy.load('en_core_web_sm')
doc = nlp(text.lower())
print([(token.lemma_, token.head, token.dep_, token.pos_) for token in doc])
print(len(doc), len(doc.text))

displacy.render(doc, style='dep', jupyter=True)

[('you', load, 'nsubj', 'PRON'), ('can', load, 'aux', 'AUX'), ('only', load, 'advmod', 'ADV'), ('load', load, 'ROOT', 'VERB'), ('onto', load, 'prep', 'ADP'), ('the', truck, 'det', 'DET'), ('truck', area, 'poss', 'NOUN'), ("'s", truck, 'case', 'PART'), ('white', area, 'amod', 'ADJ'), ('area', onto, 'pobj', 'NOUN'), ('.', load, 'punct', 'PUNCT')]
11 46


In [ ]:
# from multi_rake import Rake
# from summa import keywords
import yake

nlp = spacy.load('en_core_web_sm')
text = clean_text(get_document_by_line(CLEANED_DATASET_FILE_PATH, 155))

# def use_rake(text: str):
#     rake = Rake()
#     keywords = rake.apply(text)
#     return keywords[:30]

def use_yake(text: str):
    kw_extractor = yake.KeywordExtractor(top=20)
    keywords_info = kw_extractor.extract_keywords(text)
    keyword_groups = [keyword_info[0] for keyword_info in keywords_info if keyword_info[1] < 0.1]
    return keyword_groups

# def use_TextRank(text: str):
#     TR_keywords = keywords.keywords(text, scores=True)
#     return TR_keywords

# display(use_rake(text))
# display(use_TextRank(text))
use_yake(text)

In [ ]:
from spacy.language import Language
from spacy.lang.en import EnglishLemmatizer
from spacy.vocab import Vocab
from typing import Optional, Any, Callable, List

# class TokenWithLowerText:
#     def __init__(self, wrapped_token: spacy.tokens.Token):
#         self.__wrapped_token = wrapped_token
#         self.text = wrapped_token.text.lower()
#         self.pos_ = 'NOUN' if self.__wrapped_token.pos_ == 'PROPN' else self.__wrapped_token.pos_
#         # 92 is pos of NOUN , 96 is pos of PROPN
#         self.pos = 92 if self.__wrapped_token.pos == 96 else self.__wrapped_token.pos
#         self.orth = self.__wrapped_token.vocab.strings[self.text]
        

#     def __getattr__(self, name):
#         # print(f"{self.__wrapped_token.text}.{name} = {getattr(self.__wrapped_token, name)}")
#         return getattr(self.__wrapped_token, name)

# class LowerLemmatizer(EnglishLemmatizer):
#     def __init__(self,
#                  vocab: Vocab,
#                  lookup_tables: Any,
#                  model: Optional[Any],
#                  name: str,
#                  *,
#                  mode: str = "lookup",
#                  scorer: Optional[Callable]):
#         super().__init__(vocab, model, name, mode=mode, overwrite=True, scorer=scorer)
#         self.initialize(lookups=lemmatizer_lookup_tables)

#     def is_base_form(self, token: spacy.tokens.Token) -> bool:
#         return True if token.pos_ == 'PROPN' else super().is_base_form(token)

#     def rule_lemmatize(self, token: spacy.tokens.Token) -> List[str]:
#         return super().rule_lemmatize(TokenWithLowerText(token))

#     def lookup_lemmatize(self, token: spacy.tokens.Token) -> List[str]:
#         return super().lookup_lemmatize(TokenWithLowerText(token))

# if not Language.has_factory('lower_lemmatizer'):
#     nlp = spacy.load('en_core_web_sm')
#     lemmatizer_lookup_tables = nlp.get_pipe("lemmatizer").lookups
#     assert lemmatizer_lookup_tables is not None

#     @Language.factory(
#         "lower_lemmatizer",
#         assigns=["token.lemma"],
#         default_config={
#             "model": None,
#             "mode": "lookup",
#             "scorer": {"@scorers": "spacy.lemmatizer_scorer.v1"},
#         },
#         default_score_weights={"lemma_acc": 1.0},
#     )
#     def make_lower_lemmatizer(nlp: Language,
#                               model: Optional[Any],
#                               name: str,
#                               mode: str,
#                               scorer: Optional[Callable]
#                               ):
#         return LowerLemmatizer(
#             nlp.vocab, lemmatizer_lookup_tables, 
#             model, name, mode=mode, scorer=scorer
#         )

# text = """Axis Empires: Dai Senso! Living Rules (February 2014) 61 Design Note: Although these two countries can't be conquered, countries can be subdued through the Axis Forced Settlement Conditional Event () or an Allied Surrender Conditional Event ( and )."""
# nlp = spacy.load('en_core_web_sm')
# # nlp.replace_pipe('lemmatizer', 'lower_lemmatizer', config={"mode": "rule"})
# doc = nlp(text)
# print([token.lemma_ for token in doc])
#['sometimes', ',', 'good', 'and', 'good', 'be', 'different', 'lemma', ',', 'not', 'this', 'conditional', 'time', '.', 'the', 'same', 'go', 'for', 'rule', 'and', 'rule']
#['sometimes', ',', 'goods', 'and', 'good', 'are', 'different', 'lemma', ',', 'not', 'this', 'conditional', 'time', '.', 'the', 'same', 'go', 'for', 'rule', 'and', 'rule']

['Axis', 'Empires', ':', 'Dai', 'Senso', '!', 'live', 'rule', '(', 'February', '2014', ')', '61', 'design', 'note', ':', 'although', 'these', 'two', 'country', 'can', 'not', 'be', 'conquer', ',', 'country', 'can', 'be', 'subdue', 'through', 'the', 'Axis', 'Forced', 'Settlement', 'Conditional', 'Event', '(', ')', 'or', 'an', 'Allied', 'Surrender', 'Conditional', 'event', '(', 'and', ')', '.']
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7efc03b1f590>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7efc03b1f440>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7efc03b50e50>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7efc02c59730>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7efc02c5df50>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7efc02d0ff50>)]


In [10]:
import re
from collections import Counter, defaultdict
from typing import List, Set, Dict
from nltk.util import ngrams
# import yake
import numpy as np
from string import punctuation

regex_word_within_boundaries = re.compile(r'\b')
MIN_TOKEN_TO_BE_CONSIDERED_COMPONENT = 4
MAX_COMPONENTS = 100

ignored_words = {
        'amount', 'beginning', 'board', 'book', 'bottom', 'case', 'choice', 
        'clarification', 'clockwise', 'condition', 'cost', 'design', 'effect', 
        'end', 'example', 'face', 'front', 'game', 'left', 'middle', 'note', 'number', 
        'opponent', 'option', 'order', 'overview', 'page', 'play',
        'purpose', 'reference', 'result', 'right', 'rule', 'rulebook', 
        'section', 'set', 'setup', 'side', 'summary', 'start', 'step', 'thing',
        'type', 'tie', 'time', 'top', 'total', 'use', 'value', 'version', 'way'
        }.union(spacy.load('en_core_web_sm').Defaults.stop_words)

def find_most_common_nouns(doc: spacy.tokens.Doc) -> Dict[str, List[spacy.tokens.Token]]:
    tokens_dict = defaultdict(list)

    for token in doc:
        if len(token) >= 3 and \
            token.pos_ in {'NOUN', 'PROPN'} and \
            token.dep_ in {'nsubj', 'dobj', 'nsubjpass', 'pobj'}:
            tokens_dict[token.lemma_.lower()].append(token)
           
    return tokens_dict

def _is_token_part_of_bigram(token: spacy.tokens.Token, 
                             unigram_token: spacy.tokens.Token) -> bool:
    return token.dep_ == 'compound' and \
        token.pos_ in {'NOUN', 'PROPN'} and \
        not token.text.endswith(tuple(punctuation)) and \
        not token.text.startswith(tuple(punctuation)) and \
        token.head.i == unigram_token.i

def _find_most_relevant_ngram(doc: spacy.tokens.Doc,
                              unigrams: Dict[str, List[spacy.tokens.Token]]):
    excluded_bigrams = ignored_words.union(set(unigrams.keys()))
    bigram_associated_dict = defaultdict(set)
    for name, tokens in unigrams.items():
        for token in tokens:
            possible_bigram = doc[token.i - 1]
            if token.i > 0 and _is_token_part_of_bigram(possible_bigram, token) and \
                possible_bigram.lemma_ not in excluded_bigrams:
                bigram_associated_dict[name].add(possible_bigram.lemma_)

    return bigram_associated_dict


def _find_most_relevant_unigram_nouns(doc: spacy.tokens.Doc, nlp: spacy.Language) \
        -> Dict[str, List[spacy.tokens.Token]]:
    possible_components_info = dict(filter(lambda token: token[0] not in ignored_words and 
                                           len(token[1]) >= MIN_TOKEN_TO_BE_CONSIDERED_COMPONENT and
                                           any(token_occurrence.dep_ in {'nsubj', 'nsubjpass', 'dobj'}
                                               for token_occurrence in token[1]), 
                                           find_most_common_nouns(doc).items()))

    return possible_components_info

def _find_most_relevant_sentences(doc: spacy.tokens.Doc, unigrams: Dict[str, List[spacy.tokens.Token]]) \
        -> List[spacy.tokens.Span]:
    most_relevant_component = set(unigrams.keys())
    return [sent for sent in doc.sents if any(token.pos_ == 'VERB' for token in sent) and \
        any(token.lemma_ in most_relevant_component for token in sent)]

def get_bg_components(text: str) -> Dict[str, List[int]]:
    nlp = spacy.load('en_core_web_sm')
    original_text = text
    doc = nlp(text.lower())
    most_common_unigrams = _find_most_relevant_unigram_nouns(doc, nlp)
    most_common_bigrams = _find_most_relevant_ngram(doc, most_common_unigrams)
    most_relevant_sentences = _find_most_relevant_sentences(doc, most_common_unigrams)
    display(most_common_bigrams)
    display(len([sent for sent in doc.sents]), len(most_relevant_sentences), most_relevant_sentences)

    return most_common_unigrams

def get_doc_variance(doc: spacy.tokens.Doc, components_dict: Dict[str, List[int]]) -> float:
    '''variance measures how components interleave in the text. This could mean that rules involve
    many components and are therefore more complex. variancy is computed using `np.var` on each
    component list. the results are normalized by multiplicating for the frequency of the component.
    eventually the partial variances are summed together and the result normalized with the 
    total numbers of tokens.'''
    tokens_count = sum(len(token_list) for token_list in components_dict.values())
    return sum((len(tokens) / tokens_count) * np.var([token.i for token in tokens])
        for tokens in components_dict.values()) / len((doc))


text = get_document_by_line(CLEANED_DATASET_FILE_PATH, 148)
display(text)
components = get_bg_components(text)
display(components.keys())
print(len(components.keys()))

"ZACK & PACK For 3 6 players, age 10 & up The game is played over several rounds. Each round consists of 1. Score points The game fi nishes once a player runs out of points. GAME COMPONENTS 64 points markers PREPARATION Shuffl e truck cards into a face-down draw pile Place the wooden pieces next to the draw pile Give each player 75 points worth of markers (1x50, 2x10, 1x5). The remaining markers form the bank. The youngest player takes the 5 dice and starts the game. Receive load Each player rolls the 5 dice. Each die shows the number of wooden pieces in the corresponding colour that the player has to take. Example White die: 3 Grey die: 2 Brown die: 3 Turquoise die: blank (counts as 0) Purple die: 1 The player takes 3 white, 2 grey, 3 brown, and 1 purple pieces. Pick a truck Each player receives 2 truck cards (in a 3 or 4 player game) or 1 truck card (in a 5 or 6 player game). player places the card(s) face down in front of game , with the arrow on each card pointing towards game . Pl

defaultdict(set, {'player': {'ip'}, 'truck': {'load'}, 'area': {'loading'}})

42

36

[zack & pack for 3 6 players, age 10 & up the game is played over several rounds.,
 each round consists of 1. score points the game fi nishes once a player runs out of points.,
 game components 64 points markers preparation shuffl e truck cards into a face-down draw pile place the wooden pieces next to the draw pile give each player 75 points worth of markers (1x50, 2x10, 1x5).,
 the youngest player takes the 5 dice and starts the game.,
 receive load each player rolls the 5 dice.,
 each die shows the number of wooden pieces in the corresponding colour that the player has to take.,
 example white die: 3 grey die: 2 brown die: 3 turquoise die: blank (counts as 0) purple die: 1 the player takes 3 white, 2 grey, 3 brown, and 1 purple pieces.,
 pick a truck each player receives 2 truck cards (in a 3 or 4 player game) or 1 truck card (in a 5 or 6 player game).,
 player places the card(s) face down in front of game , with the arrow on each card pointing towards game .,
 players simultaneousl

dict_keys(['player', 'round', 'point', 'card', 'piece', 'dice', 'truck', 'area', 'space'])

9


In [23]:
nlp = spacy.load('en_core_web_sm')
print(nlp.Defaults.stop_words)
doc = nlp(get_document_by_line(CLEANED_DATASET_FILE_PATH, 130))
for sent in doc.sents:
    print(sent)

{'two', 'me', 'yourself', 'also', "'m", 'thereafter', 'cannot', 'seems', 'should', 'his', '’ll', 'noone', 'himself', 'if', 'so', 'latterly', 'itself', 'made', 'was', 'our', "n't", 'seem', 'up', 'it', 'per', 'where', 'hundred', 'ever', 'how', 'is', 'must', 'call', 'than', 'somehow', 'various', 'nowhere', 'first', 'again', 'became', 'becoming', 'hers', 'since', 'under', 'third', 'he', 'nevertheless', 'would', 'besides', 'wherein', 'have', 'upon', 'fifty', 'except', 'mine', 'ourselves', 'nine', 'get', 'while', 'whenever', 'a', 'part', 'here', "'re", 'anyhow', 'beforehand', '‘ll', 'front', 'around', 'bottom', 'move', 'him', 'most', 'along', 'there', '‘d', 'their', 'each', 'side', 'afterwards', 'behind', 'such', 'always', 'you', 'everything', 'alone', 'am', 'none', 'this', 'doing', 'already', 'sometime', 'had', 'everywhere', 'n’t', 'own', 'many', 'both', 'same', 'make', 'other', 'will', '’ve', 'about', 'anyone', "'s", 'can', 'during', 'she', 'several', '’s', 'meanwhile', 'former', 'at', 'be

In [ ]:
import itertools
from collections import Counter
from typing import List

def find_n_most_common_nouns(n, docs: List[spacy.tokens.Doc]) -> List[str]:
    docs_sets = [set(find_most_common_nouns(doc).keys())
                 for doc in docs]
    all_tokens_from_docs = itertools.chain(*docs_sets)
    tokens_counter = Counter(all_tokens_from_docs)
    return tokens_counter.most_common(n)
    

nlp = spacy.load('en_core_web_sm')
df_dataset = get_df_with_docs(CLEANED_DATASET_FILE_PATH, 100, 200)
docs = nlp.pipe(df_dataset['rulebook'].values)

find_n_most_common_nouns(80, docs)

[('player', 99),
 ('game', 97),
 ('turn', 90),
 ('end', 83),
 ('time', 83),
 ('number', 83),
 ('card', 81),
 ('order', 81),
 ('point', 73),
 ('action', 69),
 ('rule', 69),
 ('side', 64),
 ('deck', 64),
 ('table', 64),
 ('hand', 63),
 ('space', 63),
 ('front', 62),
 ('value', 61),
 ('place', 61),
 ('example', 59),
 ('pile', 56),
 ('play', 56),
 ('top', 56),
 ('type', 55),
 ('effect', 53),
 ('round', 53),
 ('marker', 52),
 ('phase', 51),
 ('board', 51),
 ('way', 50),
 ('case', 50),
 ('choice', 48),
 ('area', 47),
 ('one', 46),
 ('color', 45),
 ('box', 45),
 ('track', 44),
 ('tie', 44),
 ('right', 44),
 ('total', 42),
 ('cards', 42),
 ('face', 42),
 ('players', 41),
 ('cost', 41),
 ('step', 40),
 ('rest', 40),
 ('bottom', 40),
 ('opponent', 40),
 ('start', 38),
 ('ability', 37),
 ('symbol', 37),
 ('beginning', 37),
 ('die', 36),
 ('page', 36),
 ('result', 36),
 ('set', 35),
 ('movement', 35),
 ('condition', 34),
 ('setup', 34),
 ('token', 33),
 ('stack', 33),
 ('amount', 33),
 ('part', 33

In [ ]:
from typing import Tuple
import pandas as pd
import ast
import os
from IPython.display import clear_output

def get_rules_features(id: int, doc: spacy.tokens.Doc) -> Tuple[int, float]:
    logger.info(f'processing board game {id}')
    rulebook_len = len(doc)
    bg_components = get_bg_components(doc)
    print(bg_components)

    return 0, 0
    # rules = get_rules(text)
    # rule_count = len(rules)
    # return rule_count, len(text) / rule_count

def apply_for_rulebook_features(row, docs_dict):
    next_doc_info = next(docs_dict)
    assert next_doc_info[0] == row.id
    return pd.Series(get_rules_features(row.id, next_doc_info[1]), 
                     index=['rule_count', 'avg_rule_len'])

PROCESSED_DATASET_FILE_PATH = 'data/processed_dataset.csv' if WORKING_LOCALLY \
    else '/content/drive/My Drive/Projects/IRBoardGameComplexity/processed_dataset.csv'

# ast.literal_eval converts the family column string into a python array
df_dataset = pd.read_csv(CLEANED_DATASET_FILE_PATH, converters={ 'family': ast.literal_eval }, nrows=1)
remove_columns_prefix(df_dataset)
docs_dict = zip(df_dataset['id'].values, 
                nlp.pipe(map(clean_text, df_dataset['rulebook'].values)))

df_rules_features = df_dataset.apply(lambda x: apply_for_rulebook_features(x, docs_dict),
                                     axis='columns')
df_features = df_dataset[['averageweight', 'playingtime', 'family']].join(df_rules_features)
        
# one-hot encoding "family" field 
# from https://stackoverflow.com/questions/71401193/one-hot-encoding-in-python-for-array-values-in-a-dataframe
df_features = df_features.join(df_features.pop('family').apply('|'.join).str.get_dummies())
df_features.head()

# df_features.to_csv(PROCESSED_DATASET_FILE_PATH, header=True, index=False, mode='w')    
# if not WORKING_LOCALLY:
#     drive.flush_and_unmount()